# **Scraping web site : Hacker News**

In [ ]:
import webbrowser
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
from time import time, sleep

## **2. Schematise the HTML structure by drawing tree block**

This command opens the link of the file I inserted on my google drive.
It is also available in the .zip file if there is a problem opening it.

In [ ]:
webbrowser.open('https://drive.google.com/file/d/1GUOLmITXO78E2WoPkhtNNYQj-7r2FLwn/view?usp=sharing')

## **3. Get the HTML page in Python using the requests library**

In [ ]:
html_website = requests.get('https://news.ycombinator.com/')
soup = BeautifulSoup(html_website.text, 'html.parser')

## **4. Start parsing the page with Beautiful Soup**

### **b) Print all the title on the homepage (30 names)**

In [ ]:
titlelink = []
for title in soup.find_all('a', {'class':'titlelink'}) :
    titlelink.append(title.text)
titlelink

In [ ]:
#Let's check that we have the 30 titles of each tag
len(titlelink)

## **5. Structure your information by creating a dataclass named Post containing all the relevant information**

In [ ]:
#creation of the Post class
#Each class is defaulted to 'nan' except for the rank and the post name  

class Post():
    def __init__(self, rank, name, site = np.nan, site_url = np.nan, point = np.nan, author = np.nan, date_published = np.nan, number_comments = np.nan):
        self.rank = rank
        self.name = name
        self.site = site
        self.site_url = site_url
        self.point = point
        self.author = author
        self.date_published = date_published
        self.number_comments = number_comments

## **6. Parse the following informations: name , points , author , data published , number of comments**

In [ ]:
list_tr = soup.find('table',{'class':'itemlist'}).find_all('tr')

#size is defined as the size of the 'list_tr' - 2 because the last two 'tr' are not useful
size = len(list_tr)-2

all_posts = []


for index in range(0,size,3):
    line1 = list_tr[index] 
    line2 = line1.next_sibling   
    post = Post(
        rank = line1.find('span',{'class':'rank'}).text.replace('.',''),    
        name = line1.find('a',{'class':'titlelink'}).text )

#We have a range with a step of 3 because the information for each article is located every three 'tr' tags.
#line1 represents the first 'tr' among the blocks of 3 where the title, the name of the site and its url are found
#line2 represents the second 'tr' among the blocks of 3 where the points, the author and the date of publication are found 
    
#We perform a condition on the other classes
#If the class has content then the class takes it
    
    #Get the name of the website
    element = line1.find('a',{'class':'titlelink'})  
    
    if element :
        post.site_url = element.get('href')
        
    element = line1.find('span',{'class':'sitestr'})
    if element :
        post.site = element.text
        
    element = line2.find('span',{'class':'score'})
    if element:
        post.point=element.text.replace(' points','')
        
    element = line2.find('a',{'class':'hnuser'})
    if element:
        post.author=element.text
        
    element = line2.find('span',{'class':'age'})
    if element:
        post.date_published=element.get('title')
        
    element = line2.find(string=re.compile('comment'))
    if element:
        post.number_comments=element.replace('comments','')
        
    #Save the current post    
    all_posts.append(vars(post))

## **7. Write the data in a CSV**

In [ ]:
data = pd.DataFrame(all_posts).set_index('rank')
data.to_csv('posts.csv')
data.head()

## **8. Write code to parse the 5 first pages of HackerNews**

In [ ]:
def parseFivePages(filename='posts_5_pages.csv'):
    all_posts = []

#extraction of pages 1 to 5

    for page in range(1,6):
        html_website = requests.get(f'https://news.ycombinator.com?p={page}')
        soup = BeautifulSoup(html_website.text, 'html.parser')
        list_tr = soup.find('table',{'class':'itemlist'}).find_all('tr')
        size = len(list_tr)-2
        
        
        #rewriting the code for page 1 that will run on pages 1 to 5

        for index in range(0,size,3):
            line1 = list_tr[index]
            line2 = line1.next_sibling
            post = Post(
                rank = line1.find('span',{'class':'rank'}).text.replace('.',''),
                name = line1.find('a',{'class':'titlelink'}).text)
            
            #Get the name of the website
            element = line1.find('a',{'class':'titlelink'})
           
            #If element is not none
            if element :
                post.site_url = element.get('href')

            element = line1.find('span',{'class':'sitestr'})
            if element :
                post.site = element.text

            element = line2.find('span',{'class':'score'})
            if element:
                post.point = element.text.replace(' points','')

            element = line2.find('a',{'class':'hnuser'})
            if element:
                post.author = element.text

            element = line2.find('span',{'class':'age'})
            if element:
                post.date_published = element.get('title')

            element = line2.find(string = re.compile('comment'))
            if element:
                post.number_comments = element.replace('comments','')
                
            #Save the current post    
            all_posts.append(vars(post))
          
        
    #creation of the dataframe and the CSV file for the 5 pages
    data = pd.DataFrame(all_posts)
    data = data.set_index('rank')
    data.to_csv(filename)

In [ ]:
parseFivePages()

## **9. Think about a way to run your code every hour**

In [ ]:
one_hours  = 60 * 60  

#one_hours is defined in seconds
#in an hour there are 60 minutes that have 60 seconds

while True:
    sleep(one_hours) 
    parseFivePages('every_hours.csv') #creation of the CSV file witch run each hours